<a href="https://colab.research.google.com/github/Ismail-Armutcu/BlockChain-CryptoCurrencyTechnologies/blob/main/Block%20Chain%20Analysis/BitcoinBlockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#METU CSES 519 BlockChain and CryptoCurrency Technologies
##Bitcoin Block Analysis
- İsmail Hakkı Armutcu
- 2374395

Preamble

In [152]:
import requests
from pprint import pprint  # Import pprint for pretty printing
import json
import binascii
import re

Utility functions

In [162]:
def get_block_details(block_hash):
    url = f"https://blockchain.info/rawblock/{block_hash}"
    response = requests.get(url)
    block = response.json()  # Convert the JSON response to a dictionary
    return block

def get_bch_block_details(block_hash):
    # URL of the FullStack.cash API endpoint
    url = "https://api.fullstack.cash/v5/blockchain/getBlock/"

    # Headers to indicate that we're sending JSON data
    headers = {
        'Content-Type': 'application/json'
    }

    # JSON payload with the block hash and verbosity level
    data = {
        'blockhash': block_hash,
        'verbosity': 2
    }

    # Send the POST request
    response = requests.post(url, headers=headers, json=data)

    # Check if the request was successful
    if response.status_code == 200:
        # Return the parsed JSON data
        return response.json()
    else:
        # Print error and return None if the API call failed
        print(f"Failed to fetch data: HTTP {response.status_code} - {response.text}")
        return None


def extractMessages(hex_string):

    byte_data = binascii.unhexlify(hex_string)


    # Define ASCII printable character ranges (space to ~)
    printable_ascii_range1 = (65, 90)
    printable_ascii_range2 = (97, 122)
    # Buffer to collect printable characters
    result = []
    current_printable_sequence = []

    # Iterate over each byte and check if it is a printable ASCII character
    for byte in byte_data:
        if printable_ascii_range1[0] <= byte <= printable_ascii_range1[1] or printable_ascii_range2[0] <= byte <= printable_ascii_range2[1]:
            current_printable_sequence.append(chr(byte))
        else:
            # If the current byte is not printable and there is an ongoing sequence,
            # save it if it has two or more characters
            if len(current_printable_sequence) > 1:
                result.append(''.join(current_printable_sequence))
            current_printable_sequence = []

    # Check if there is a remaining printable sequence at the end of the data
    if len(current_printable_sequence) > 1:
        result.append(''.join(current_printable_sequence))
    return result


#Fetch the Corresponding Bitcoin block

In [78]:
block_hash = '00000000000000e25266d700ea597112a7adf393802db132f1ae7caee7313bdd'  # Hash of the block number 237439
block_details = get_block_details(block_hash)

## 1- Hash Puzzle Solution
- Hash of the block number 237439 is: 00000000000000e25266d700ea597112a7adf393802db132f1ae7caee7313bdd

- It starts with 8*14 =  112 zero bits

In [83]:
blockHeader = block_details.copy()
del blockHeader['tx']
hash = blockHeader.get('hash')
print("Hash of block number 237439 is:",hash)
print("Block Header")
pprint(blockHeader)
#14 zero bytes = 14*8 = 112 bits

Hash of block number 237439 is: 00000000000000e25266d700ea597112a7adf393802db132f1ae7caee7313bdd
Block Header
{'bits': 436305897,
 'block_index': 237439,
 'fee': 38920000,
 'hash': '00000000000000e25266d700ea597112a7adf393802db132f1ae7caee7313bdd',
 'height': 237439,
 'main_chain': True,
 'mrkl_root': '82bfde6739590e6c56bde7093f63fb086da23707aceaa38ec36619b6552e9b3d',
 'n_tx': 486,
 'next_block': ['00000000000001722d317505113974aa5c90337488b353da4cd8ecc3fd31f170'],
 'nonce': 1259351560,
 'prev_block': '00000000000000e029b20ad0288ec0e4cfc65e62c09333247a65c1c506bfbc8c',
 'size': 215070,
 'time': 1369277732,
 'ver': 2,
 'weight': 860280}


#2-CoinBase Transaction
$Block Reward$: The value field in the output (out) section shows 2538920000 satoshis, which equates to 25.3892 BTC.At the time this block was mined, the block reward was still at 25 BTC, indicating that the transaction fees included amounted to 0.3892 BTC.

$Fee Reward$: There is no explicit fee associated with the coinbase transaction itself.

$Extra Nonce$: Miner pool added a message in the input script: Mined by BTC Guild

In [129]:
coinBase = block_details['tx'][0]
print("CoinBase Transaction")
pprint(coinBase)
script = coinBase['inputs'][0]['script']
print("Special Message in Script:",extractMessages(script))

CoinBase Transaction
{'block_height': 237439,
 'block_index': 237439,
 'double_spend': False,
 'fee': 0,
 'hash': '229ecce9a5aeef9552c7310bbb695051eb8b3379e6fd62018f979f536a334873',
 'inputs': [{'index': 0,
             'prev_out': {'n': 4294967295,
                          'script': '',
                          'spending_outpoints': [{'n': 0,
                                                  'tx_index': 4056125998265330}],
                          'spent': True,
                          'tx_index': 0,
                          'type': 0,
                          'value': 0},
             'script': '037f9f030400000150124d696e656420627920425443204775696c64080000006100003002',
             'sequence': 4294967295,
             'witness': ''}],
 'lock_time': 0,
 'out': [{'addr': '14cZMQk89mRYQkDEj8Rn25AnGoBi5H6uer',
          'n': 0,
          'script': '76a91427a1f12771de5cc3b73941664b2537c15316be4388ac',
          'spending_outpoints': [{'n': 0, 'tx_index': 2663907566807796}],
     

#3-Wallets
- There are 486 transactions in the block
- Number of unique addresses in block: 1005
- Number of addresses with prefix 1: 1005 -> 100%
- Number of addresses with prefix 3: 0 -> 0%
- Number of addresses with prefix bc1: 0 -> 0%

Bech32 addresses that start with bc1 was introduced later than the mining date of this block which is May 23 2013. Hence there are no addresses that start with prefix bc1

P2SH addresses that start with 3 was introduced in 2012, one year before this block. However due to the decentralized nature of bitcoin, changes and updates progate slowly. Also software support for new updates takes time to mature. Hence there was no address that start with 3.

All the addresses start with 1 which is P2PKH.



In [138]:
transactions = block_details.get("tx")
print("Number of transactions: ",len(transactions) )
# Set to store unique addresses
unique_addresses = set()
for tx in transactions:
# Extract addresses from inputs
  for input_ in tx['inputs']:
    if 'addr' in input_['prev_out']:
      unique_addresses.add(input_['prev_out']['addr'])

# Extract addresses from outputs
  for output in tx['out']:
    if 'addr' in output:
      unique_addresses.add(output['addr'])

# Print all unique addresses found
print("Number of unique addresses in block:", len(unique_addresses))

addressWithPrefix1 = []
addressWithPrefix3 = []
addressWithPrefixbc1 = []

for addr in unique_addresses:
  if addr.startswith('1'):
    addressWithPrefix1.append(addr)
  elif addr.startswith('3'):
    addressWithPrefix1.append(addr)
  elif addr.startswith('bc1'):
    addressWithPrefixbc1.append(addr)

print("Number of addresses with prefix 1:",len(addressWithPrefix1))
print("Number of addresses with prefix 3:",len(addressWithPrefix3))
print("Number of addresses with prefix bc1:",len(addressWithPrefixbc1))





Number of transactions:  486
Number of unique addresses in block: 1005
Number of addresses with prefix 1: 1005
Number of addresses with prefix 3: 0
Number of addresses with prefix bc1: 0


#4-Vanity Addresses

Address 52: 1$CRACk$biJSxfDaLNEoaNsHjNtU4KttwHyo

Address 71: 1$dice$2pxmRZrtqBVzixvWnxsMa7wN2GCK

Address 210: 1$LA$621LfnJn2sNsGXPdCv5SxMkFyAGPokQ

Address 230: 1$dice$2vQoUkQwDMbfDACM1xz6svEXdhYb

Address 497: 1L4$EThM$6x3Rd2PjNbs1U136FpMq4Gmo3fJ
  

In [157]:
# Regular expression to find sequences of two or more alphabetic characters
pattern = re.compile(r'[A-Za-z]{2,}')

# Check each address and print parts with two or more back-to-back alphabetic characters
i = 0
for address in unique_addresses:
    match = pattern.search(address)
    if match:
        print(f'Address {i}: {address}')
        print(f'   Matched Sequence Right After 1: {match[0]}')

    i = i+1

Address 0: 1LEDpiLdSA9dCY9CDjq3YDAz52HiN6bF2J
   Matched Sequence Right After 1: LEDpiLdSA
Address 1: 13N7TNJpdp2mzJf6pwXU8n3Tf8bUkkirL9
   Matched Sequence Right After 1: TNJpdp
Address 2: 1H1FS8WLU8BBEDNfXWh1zKXeQxZvCyYE4x
   Matched Sequence Right After 1: FS
Address 3: 1F16aJRuSdz1CBLChmwmHLaaz3NuHaFtog
   Matched Sequence Right After 1: aJRuSdz
Address 4: 1CzAHRSS1CAVQ3FTUENgSJTi5Bk2EFCrD
   Matched Sequence Right After 1: CzAHRSS
Address 5: 1GE5zSjJBuhGkxYxM3x5LmTdnNyt1p2sVf
   Matched Sequence Right After 1: GE
Address 6: 131WL5DMbsYb3poHUByC11CEgfSMnftpeK
   Matched Sequence Right After 1: WL
Address 7: 15NRAfLoJ8Sfob6Lb2PqA8pZxAbBGCkXYX
   Matched Sequence Right After 1: NRAfLoJ
Address 8: 1K5CkmFESUDas2JgzKDu4rqZDEhXuUtGZk
   Matched Sequence Right After 1: CkmFESUDas
Address 9: 15CGaKa7GVnNXGdbkxKiw2CM7XiJrq4NmX
   Matched Sequence Right After 1: CGaKa
Address 10: 1LD6jejzJNS5YTjAXQN5ycknhhuC9XeLvG
   Matched Sequence Right After 1: LD
Address 11: 1B7bM5CTfcSn3qwdhMYfJUzc5ct

#5-Hidden Messages
- ['Mined', 'by', 'BTC', 'Guild']

I could only detect the coinBase message

In [164]:
scripts = [] ##Extract all the scripts
for tx in transactions:
  for input_ in tx['inputs']:
    scripts.append(input_['script'])
    scripts.append(input_['prev_out'].get('script'))
  for output_ in tx['out']:
    scripts.append(output_['script'])

for script in scripts:
  message = extractMessages(script)
  if len(message) >2:
    print(message)



['Mined', 'by', 'BTC', 'Guild']
['qo', 'Uu', 'lu']
['MU', 'ao', 'WK', 'fD', 'em']
['MT', 'vH', 'uT']
['BS', 'dr', 'br', 'wZZ']
['cY', 'ljpB', 'dV', 'mW', 'dc', 'Lx', 'kZ', 'vI']
['FJ', 'nU', 'HS', 'Lx', 'kZ', 'vI']
['gef', 'IxI', 'mH', 'wm', 'ap']
['IN', 'ry', 'OkNJ', 'wm', 'ap']
['Hs', 'RF', 'mD', 'ihLr', 'EEVj', 'EA', 'WN']
['me', 'RN', 'QZ', 'Ox', 'hVB', 'Sn']
['iy', 'KQ', 'bB']
['vl', 'DSy', 'qV']
['lS', 'NB', 'Py', 'uY']
['Wa', 'Kj', 'GN', 'NyT', 'Vc']
['ly', 'NI', 'Kk']
['jS', 'wM', 'Zjr', 'ly', 'sd']
['WF', 'jK', 'cu', 'wE', 'zJ']
['eb', 'Ly', 'WFJ', 'OF']
['WP', 'BE', 'RJ', 'oi']
['bfQ', 'kU', 'pV', 'aG', 'Uw', 'hK', 'Mi', 'Onw', 'IMfj']
['lQ', 'pR', 'xn', 'CS', 'wT']
['muC', 'oY', 'Lh', 'uf']
['Ze', 'Ho', 'vLn', 'Jf', 'li']
['so', 'VU', 'iq']
['Al', 'zp', 'VG', 'Uz']
['Qb', 'Op', 'Ys', 'WEP', 'Nb']
['ij', 'kq', 'oS']
['ER', 'LW', 'eP', 'mT']
['Fx', 'tic', 'rD', 'AN', 'KId']
['Ai', 'Uq', 'ov', 'rXp', 'Yj', 'RQQ', 'fd']
['lmyO', 'et', 'zjT']
['Ls', 'XV', 'Wa', 'Ia']
['LJ', 'BI',

#6-Bitcoin CASH Block 666666 Hidden Messages

In [196]:
block_hash = '000000000000000000bf2f81e258d4ad8d0b4a8fb2f2f1b65b11a3d5d3916a01'
block_details = get_bch_block_details(block_hash) ## Fetch bch block 666666
block_header = block_details.copy()
del block_header['tx']
pprint(block_header)

{'bits': '1804ae3d',
 'chainwork': '00000000000000000000000000000000000000000157d1f68e5630a299a63188',
 'confirmations': 178665,
 'difficulty': 234903763423.1902,
 'hash': '000000000000000000bf2f81e258d4ad8d0b4a8fb2f2f1b65b11a3d5d3916a01',
 'height': 666666,
 'mediantime': 1608446918,
 'merkleroot': 'd9c7c84aa146b4a560ec254feac91dae7e4ec2546a4d51ba36a254e57499233f',
 'nTx': 86,
 'nextblockhash': '000000000000000003ca89590a2bf542559e89e93a188dc172cb238e551cb77f',
 'nonce': 1545350804,
 'previousblockhash': '0000000000000000014f29cfac480cf3f696728a9b31c181c0d77a746d50f83b',
 'size': 83121,
 'time': 1608450122,
 'version': 536870912,
 'versionHex': '20000000'}


 # Bitcoin CASH Block 666666 Hidden Messages

 - 'BCH', 'IS', 'THE', 'REAL', 'BITCOIN'


 - 'The', 'greatest', 'glory', 'in', 'living', 'lies', 'not', 'in', 'never', 'falling', 'but', 'in', 'rising', 'every', 'time', 'we', 'fall', 'Nelson', 'Mandela'

 - 'Distance', 'unites', 'missing', 'beats', 'of', 'two', 'hearts', 'in', 'love'

In [192]:
transactions = block_details['tx']
transactions.pop(0)
hexData = []
for tx in transactions:
  if 'coinbase' in tx:
    hexData.append(tx['coinbase'])
  else:
    for vin in tx['vin']:
      hexData.append(vin['scriptSig']['hex'])
    for vout in tx['vout']:
      hexData.append(vout['scriptPubKey']['hex'])
for hex in hexData:
  print(extractMessages(hex))



['TU', 'BA', 'di', 'ml']
[]
[]
['OU']
['KNo']
['TQl', 'bK', 'CWb', 'AJ', 'NCabqI']
['pFvu', 'Ck', 'Sr']
[]
[]
['AG', 'bM', 'jiKV', 'ALiR', 'ot', 'am', 'rP']
['ln']
[]
['fw']
[]
['dG', 'tK']
[]
[]
['qWx', 'YI']
[]
['iT']
[]
[]
[]
['AG', 'ue', 'zY', 'ALiR', 'ASa', 'PW', 'Pb', 'tqS']
['ly', 'JZ', 'XK', 'AG', 'Nd', 'uQ', 'vy', 'yy', 'xALiR', 'Ul', 'Ur', 'fmJS']
[]
[]
['Wm', 'QAN']
['Ai', 'KK', 'VA', 'cP']
['LM', 'lL']
['Mg', 'rL', 'QH', 'wR', 'WaR']
['sp', 'hj', 'sW', 'WaR']
[]
[]
['WV']
[]
['Bg', 'Xb', 'bUD', 'cW', 'si']
['oX']
[]
['AG', 'Qq', 'jmX', 'Qi', 'tk', 'Hq', 'ALiR', 'Bj']
['jH']
[]
['lA', 'nz']
['DQ', 'ea', 'SA', 'ns']
['RH', 'bb']
['MH']
['BP']
['AG', 'ALiR', 'iPbm', 'Yf']
['wQDt']
[]
['fj', 'Jug', 'gFAGoT', 'xK']
[]
['bSv', 'kv']
['oD', 'Ab', 'WO', 'Th', 'OFF', 'ip']
['HW', 'dU', 'WO', 'Th', 'OFF', 'ip']
['WO', 'Th', 'OFF', 'ip']
['Kt']
['ag', 'zy', 'AG', 'ENO', 'QN', 'nq', 'qn', 'ALiR', 'gt', 'wY', 'oS']
['hU', 'zV', 'AG', 'Tt', 'yL', 'ALiR', 'VnKd']
['gf', 'EG', 'AG', 'Pz', 